import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "ssd_images_1_to_17_new.csv"
df = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr = []
main_labels = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in df.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr.append(sequence_of_frames)
        main_labels.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr))
print("Number of label:", len(main_labels))

In [4]:
!pip install opencv-python

In [24]:
import cv2

In [ ]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "ssd_images_1_to_17_new.csv"
df = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr = []
main_labels = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in df.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr.append(sequence_of_frames)
        main_labels.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr))
print("Number of label:", len(main_labels))

from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers,models
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, LSTM, Dense, Dropout, TimeDistributed, Reshape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras import models,layers

frames=12
height=100
width=96
channels=3
input_shape = (frames, height, width, channels)
num_classes=6

def create_3dcnn(input_shape, num_classes):
    model = Sequential()

    # Convolutional layers
    model.add(Conv3D(64, (3, 3, 3), activation='relu', padding="same", input_shape=input_shape))
    print(model.output_shape)
    model.add(MaxPooling3D((2, 2, 2), padding='same'))  

    model.add(Conv3D(128, (3, 3, 3), activation='relu', padding="same"))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Conv3D(256, (3, 3, 3), activation='relu', padding="same"))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Conv3D(512, (3, 3, 3), activation='relu', padding="same"))
    model.add(MaxPooling3D((2, 2, 2), padding='same'))

    model.add(Dropout(0.5))
    
    # Flatten the output for fully connected layers
    model.add(Flatten())

    # Fully connected layers
    model.add(Dense(1024, activation='relu')) 
    model.add(Dropout(0.5))  
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax')) 
    
    return model

model = create_3dcnn(input_shape, num_classes)

main_arr = np.array(main_arr)
main_labels = np.array(main_labels)

frames=12
height=100
width=96
channels=3
input_shape = (frames, height, width, channels)
num_classes=6

#main_arr=(main_arr/255.0)
l=LabelEncoder()
main_labels_encoded=l.fit_transform(main_labels)

#x_train,x_val,y_train,y_val=train_test_split(main_arr,main_labels_encoded,test_size=0.2,random_state=42)

kf = KFold(n_splits=3, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in kf.split(main_arr):
    x_train, x_val = main_arr[train_index], main_arr[val_index]
    y_train, y_val = main_labels[train_index], main_labels[val_index]

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history=model.fit(x_train, y_train, epochs=35, batch_size=12, verbose=1)
    #model.save('3DCNN_model.h5')

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(x_val, y_val)
    cv_scores.append(test_accuracy)

# Print the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("Mean Accuracy:", np.mean(cv_scores))


model.save('3DCNN_model.h5')
print("the model saved successfully")

Number of sequences: 1364
Number of label: 1364
(None, 12, 100, 96, 64)
Epoch 1/35
76/76 [==============================] - 21659s 285s/step - loss: 7.7143 - accuracy: 0.1771
Epoch 2/35
76/76 [==============================] - 1119s 15s/step - loss: 1.8392 - accuracy: 0.2090
Epoch 3/35
76/76 [==============================] - 1121s 15s/step - loss: 1.7717 - accuracy: 0.1771
Epoch 4/35
76/76 [==============================] - 1117s 15s/step - loss: 1.7207 - accuracy: 0.2376
Epoch 5/35
76/76 [==============================] - 1118s 15s/step - loss: 1.7188 - accuracy: 0.2728
Epoch 6/35
76/76 [==============================] - 1120s 15s/step - loss: 1.5849 - accuracy: 0.3520
Epoch 7/35
76/76 [==============================] - 1115s 15s/step - loss: 1.4008 - accuracy: 0.4488
Epoch 8/35
76/76 [==============================] - 1118s 15s/step - loss: 1.3795 - accuracy: 0.4532
Epoch 9/35
76/76 [==============================] - 1120s 15s/step - loss: 1.2370 - accuracy: 0.4906
Epoch 10/35
76/76

In [11]:
import numpy as np
import cv2
import pandas as pd

# Load CSV file
csv_path = "ssd_images_19_to_24(test_data).csv"
df = pd.read_csv(csv_path)

# Define parameters
num_frames = 12
height = 100
width = 96
channels = 3

# Create an empty NumPy array with the specified shape
sequence_of_frames = np.zeros((num_frames, height, width, channels))
main_arr_test = []
main_labels_test = []

# Iterate through each row in the DataFrame
cnt = 0
for index, row in df.iterrows():
    # Load the image file
    img_path = row['filepath']
    frame_data = cv2.imread(img_path)
    frame_data = cv2.cvtColor(frame_data, cv2.COLOR_BGR2RGB)
    
    # Store frame data in sequence_of_frames array
    sequence_of_frames[cnt] = frame_data
    cnt += 1
    
    # If sequence_of_frames array is filled, append it to main_arr and reset sequence_of_frames
    if cnt >= num_frames:
        main_arr_test.append(sequence_of_frames)
        main_labels_test.append(row['label'])
        cnt = 0
        sequence_of_frames = np.zeros((num_frames, height, width, channels))

# Print the number of sequences stored in main_arr
print("Number of sequences:", len(main_arr_test))
print("Number of label:", len(main_labels_test))

Number of sequences: 528
Number of label: 528


In [4]:
from tensorflow.keras.models import load_model

model=load_model("3DCNN_model.h5")

model.summary()

In [12]:
main_arr_test=np.array(main_arr_test)
main_labels_test=np.array(main_labels_test)

In [10]:
main_labels_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [13]:

# Make predictions
predictions = model.predict(main_arr_test)
print("The predictions are:", predictions)

The predictions are: [[0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]
 [0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]
 [0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]
 ...
 [0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]
 [0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]
 [0.18409248 0.17770661 0.18395545 0.1806123  0.08882727 0.18480594]]


In [14]:
predictions=np.array(predictions)

In [17]:
predictions

array([[0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594],
       [0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594],
       [0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594],
       ...,
       [0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594],
       [0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594],
       [0.18409248, 0.17770661, 0.18395545, 0.1806123 , 0.08882727,
        0.18480594]], dtype=float32)

In [18]:
y_pred=[]
for i in predictions:
    y_pred.append(np.argmax(i))

In [15]:
y_pred=np.argmax(predictions)

In [20]:
y_pred=np.array(y_pred)

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
test_accuracy=accuracy_score(y_pred,main_labels_test)
#print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Test Accuracy: 0.18181818181818182


In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('3DCNN_model_complex.h5')

#predictions = loaded_model.predict(new_data)


loaded_model.summary()